<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


## 🛠️ 步骤 0：环境准备与依赖安装

### 📦 安装核心依赖库

在开始本教程之前，我们需要安装以下核心库：

- **`langgraph`**：用于构建多节点、状态驱动的 AI 代理工作流
- **`langfuse`**：提供大模型应用的可观测性和评估功能  
- **`langchain`** 系列：用于 LLM 应用开发的核心框架
- **`datasets`**：Hugging Face 的数据集处理库

```bash
# 安装命令将在下方代码单元格中执行
```

<!-- CALLOUT_START type: "info" emoji: "⚠️" -->
**📌 重要提示：** 
- 本教程使用 **Langfuse Python SDK v3**，它提供了更好的性能和新特性
- 如果您仍在使用 Python SDK v2，请参考我们的[旧版 LangGraph 集成指南](https://github.com/langfuse/langfuse-docs/blob/662509b3296daddcddb292f14b10a62e7c39407d/pages/docs/integrations/langchain/example-langgraph-agents.md#L4)进行升级
- 建议在虚拟环境中运行本教程以避免依赖冲突
<!-- CALLOUT_END -->

### 🔧 环境要求

- **Python 版本**：3.8 或更高版本
- **操作系统**：支持 Windows、macOS、Linux
- **网络**：需要访问 OpenAI API 和 Langfuse 服务

In [ ]:
# 📦 安装所需的Python包
# 使用魔法命令 %pip 在Jupyter环境中安装依赖库

%pip install langfuse langchain langgraph langchain_openai langchain_community langchain_huggingface

# 各库功能说明：
# - langfuse: LLM应用的可观测性和评估平台
# - langchain: 大语言模型应用开发框架
# - langgraph: 基于langchain的图形化工作流构建工具  
# - langchain_openai: OpenAI模型的langchain集成
# - langchain_community: 社区贡献的langchain扩展
# - langchain_huggingface: Hugging Face模型的langchain集成

## 🔑 步骤 1：配置 API 密钥和环境变量

### 获取 Langfuse API 密钥

在开始使用 Langfuse 之前，您需要获取 API 访问凭证：

#### 方案一：使用 Langfuse Cloud（推荐）
1. 访问 [Langfuse Cloud](https://cloud.langfuse.com) 并注册账户
2. 创建新项目或选择现有项目
3. 在项目设置页面获取以下密钥：
   - `LANGFUSE_PUBLIC_KEY`：以 `pk-lf-` 开头的公钥
   - `LANGFUSE_SECRET_KEY`：以 `sk-lf-` 开头的私钥

#### 方案二：自托管 Langfuse
如果您选择自托管部署，请按照 [Langfuse 自托管文档](https://langfuse.com/docs/deployment/self-host) 进行配置。

### 获取 OpenAI API 密钥

1. 访问 [OpenAI 平台](https://platform.openai.com/)
2. 注册账户并完成身份验证
3. 在 API 密钥页面创建新的 API 密钥
4. 确保账户有足够的余额用于 API 调用

### 🔐 安全提醒

- **请勿将 API 密钥硬编码在代码中**
- **生产环境建议使用环境变量或密钥管理系统**
- **定期轮换密钥以提高安全性** 

In [ ]:
import os

# 🔑 配置 Langfuse API 凭证
# 从项目设置页面获取密钥：https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."  # 替换为您的公钥
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."  # 替换为您的私钥

# 🌍 设置 Langfuse 服务器地址
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧洲区域
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 美国区域（可选）

# 🤖 配置 OpenAI API 密钥
os.environ["OPENAI_API_KEY"] = "sk-proj-..."  # 替换为您的 OpenAI API 密钥

# 💡 提示：在生产环境中，建议通过以下方式设置环境变量：
# 1. 使用 .env 文件配合 python-dotenv 库
# 2. 在操作系统层面设置环境变量
# 3. 使用云平台的密钥管理服务

<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Example - Trace and Evaluate LangGraph Agents" description: "This guide shows how to evaluate LangGraph Agents with Langfuse using online and offline evaluation methods." category: "Integrations" -->

# LangGraph 代理追踪与评估完整指南

## 📖 教程概述

在本教程中，我们将深入学习如何使用 [Langfuse](https://langfuse.com)（一个强大的大模型可观测性平台）与 [Hugging Face Datasets](https://huggingface.co/datasets)，来**全面监控 [LangGraph 代理](https://github.com/langchain-ai/langgraph) 的执行过程（traces）**并**科学评估其性能表现**。

## 🎯 学习目标

本指南将帮助您掌握将 AI 代理快速且可靠地部署到生产环境所需的核心技能：
- **在线评估**：实时监控生产环境中的代理表现
- **离线评估**：使用基准数据集进行系统性测试

💡 **延伸阅读**：想了解更多 LLM 评估策略和最佳实践，请参阅我们的[详细博文](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

## 🔍 为什么 AI 代理评估如此重要？

在 AI 代理开发过程中，评估是确保系统质量的关键环节：

- **🐛 问题诊断**：当代理任务执行失败或结果不理想时，能够快速定位问题根源
- **📊 性能监控**：实时追踪系统的成本消耗、响应延迟等关键指标
- **🔄 持续改进**：通过用户反馈和评估数据，不断提升代理的可靠性与安全性
- **🚀 生产就绪**：确保代理在真实环境中能够稳定运行


### 🔗 连接验证与客户端初始化

设置完环境变量后，我们需要初始化 Langfuse 客户端并验证连接。

**核心概念解释：**
- **`get_client()`**：Langfuse 提供的便捷函数，会自动读取环境变量中的凭证
- **客户端实例**：用于与 Langfuse 服务器通信的对象
- **连接验证**：确保 API 密钥正确且网络连接正常

In [ ]:
# 📡 导入 Langfuse 客户端并建立连接
from langfuse import get_client
 
# 🔧 初始化 Langfuse 客户端
# get_client() 会自动从环境变量中读取 API 凭证
langfuse = get_client()
 
# ✅ 验证 API 连接和身份认证
# auth_check() 方法会测试与 Langfuse 服务器的连接
if langfuse.auth_check():
    print("✅ Langfuse 客户端连接成功！API 认证通过")
    print("🎯 现在可以开始追踪和评估 LLM 应用了")
else:
    print("❌ 认证失败！请检查以下项目：")
    print("   1. API 密钥是否正确设置")
    print("   2. 服务器地址是否正确")
    print("   3. 网络连接是否正常")

## 🧪 步骤 2：构建第一个 LangGraph 代理并验证追踪功能

### 💡 什么是追踪（Tracing）？

在 LLM 应用开发中，**追踪（Tracing）**是指记录应用程序执行过程中的详细信息：
- **执行路径**：代理执行了哪些步骤
- **性能指标**：每个步骤的耗时、令牌消耗等
- **输入输出**：每个环节的输入和输出内容
- **错误信息**：出现问题时的详细错误日志

### 🎯 本节目标

我们将创建一个简单的问答代理来验证 Langfuse 追踪功能是否正常工作。

**技术要点：**
- 使用 **LangGraph** 构建状态驱动的代理工作流
- 通过 **CallbackHandler** 实现自动追踪
- 在 Langfuse 仪表板中查看执行记录

🔍 **运行成功标志**：如果配置正确，您将在 [Langfuse 追踪仪表板](https://cloud.langfuse.com/traces) 中看到详细的执行日志和性能指标。

In [ ]:
# 🚀 构建简单的 LangGraph 问答代理

# 📦 导入必要的类型和工具
from typing import Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# 🔧 定义代理的状态结构
class State(TypedDict):
    # messages 字段存储对话历史，类型为列表
    # Annotated[list, add_messages] 定义了状态更新的方式：
    # - list: 数据类型为列表
    # - add_messages: 更新时追加消息而不是覆盖（保持对话历史）
    messages: Annotated[list, add_messages]

# 🏗️ 创建状态图构建器
# StateGraph 是 LangGraph 的核心类，用于构建状态驱动的工作流
graph_builder = StateGraph(State)

# 🤖 初始化 OpenAI 语言模型
# - model: 使用 GPT-4o 模型（性能强大且成本适中）
# - temperature: 设置为 0.2，输出相对稳定但保持一定创造性
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

# 💬 定义聊天机器人节点函数
def chatbot(state: State):
    """
    聊天机器人的核心逻辑
    
    参数:
        state: 当前的对话状态，包含消息历史
        
    返回:
        包含新消息的字典，会被自动合并到状态中
    """
    # 调用 LLM 处理当前所有消息，并返回回复
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# 🔗 构建工作流图结构
# 1. 添加节点：每个节点代表一个工作单元（通常是 Python 函数）
graph_builder.add_node("chatbot", chatbot)

# 2. 设置入口点：告诉图从哪个节点开始执行
graph_builder.set_entry_point("chatbot")

# 3. 设置结束点：定义工作流的终止条件
graph_builder.set_finish_point("chatbot")

# ⚙️ 编译图以获得可执行的代理
# compile() 方法将图定义转换为可运行的 CompiledGraph 对象
graph = graph_builder.compile()

print("✅ 简单问答代理构建完成！")
print("🔧 代理架构：输入 → ChatBot节点 → 输出")
print("📝 支持功能：基本问答、上下文理解")

In [ ]:
# 🔍 启用 Langfuse 追踪并运行代理

# 📡 导入 Langfuse 的 LangChain 回调处理器
from langfuse.langchain import CallbackHandler

# 🎯 初始化 Langfuse 追踪处理器
# CallbackHandler 会自动捕获 LangChain/LangGraph 的执行信息
langfuse_handler = CallbackHandler()

print("🚀 开始运行代理并启用 Langfuse 追踪...")
print("❓ 用户问题：What is Langfuse?")
print("📊 追踪信息将发送到 Langfuse 平台")
print("-" * 50)

# 🏃 运行代理并启用追踪
# stream() 方法允许实时接收代理的执行结果
for step_result in graph.stream(
    # 输入：包含用户消息的状态
    {"messages": [HumanMessage(content="What is Langfuse?")]},
    # 配置：启用 Langfuse 回调处理器进行追踪
    config={"callbacks": [langfuse_handler]}
):
    print(f"📥 代理执行步骤: {step_result}")

print("-" * 50)
print("✅ 代理执行完成！")
print("🔗 请访问 Langfuse 仪表板查看详细追踪信息")
print("📍 链接: https://cloud.langfuse.com/traces")

### 🔍 验证追踪功能：查看 Langfuse 仪表板

#### 📊 如何检查追踪记录

运行上述代码后，请按以下步骤验证追踪功能：

1. **访问仪表板**：打开 [Langfuse 追踪仪表板](https://cloud.langfuse.com/traces)
2. **查找记录**：在追踪列表中找到刚才的执行记录
3. **分析数据**：点击记录查看详细的执行信息

#### 🔬 追踪记录包含什么信息？

在 Langfuse 中，您将看到以下重要信息：

- **📝 Spans（跨度）**：每个执行步骤的详细记录
- **📋 Logs（日志）**：执行过程中的日志信息  
- **⏱️ 时间戳**：每个步骤的精确执行时间
- **💰 成本信息**：API 调用的令牌消耗和费用
- **📊 性能指标**：延迟、吞吐量等关键指标

#### 📸 Langfuse 中的示例追踪截图

![Langfuse 中的示例追踪](https://langfuse.com/images/cookbook/example-langgraph-evaluation/first-example-trace.png)

💡 **小提示**：追踪记录可能需要几秒钟才能在仪表板中显示，请稍作等待。

🔗 _[查看示例追踪记录](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/ed0970b5-b251-4b85-9023-c0ed81462510?timestamp=2025-03-20T13%3A44%3A44.381Z&display=details&observation=0731595f-06e4-4f5a-b535-6e09677a752d)_

## 🔬 步骤 3：构建并观测复杂的邮件处理代理

### 🎯 进阶实战：真实业务场景模拟

既然已确认基础追踪功能有效，现在我们来构建一个更加复杂且贴近实际业务场景的代理系统。

### 📧 业务场景：智能邮件管理助手

我们将创建一个模拟 **蝙蝠侠管家阿尔弗雷德** 的邮件处理代理，具备以下功能：

#### 🔧 核心功能模块
- **📬 邮件接收**：读取和解析邮件内容
- **🔍 垃圾邮件识别**：智能判断邮件是否为垃圾邮件
- **🗂️ 自动分类**：对合法邮件进行分类处理
- **✍️ 回复起草**：为重要邮件生成回复草稿
- **📢 通知主人**：向韦恩先生汇报重要邮件

#### 📊 追踪的高级指标

通过这个复杂代理，我们将观察以下关键指标：
- **💰 成本追踪**：详细的令牌消耗和 API 费用
- **⏱️ 性能分析**：每个处理步骤的耗时分布
- **🔄 工作流路径**：代理的决策逻辑和执行路径
- **❌ 错误监控**：异常情况的捕获和分析

### 🏗️ 技术架构特点

- **状态驱动**：使用 LangGraph 的状态管理机制
- **条件分支**：根据邮件类型执行不同的处理逻辑
- **多节点协作**：模拟真实的业务处理流程

In [ ]:
# 📦 导入构建复杂代理所需的库

import os  # 操作系统接口，用于环境变量管理
from typing import TypedDict, List, Dict, Any, Optional  # 类型注解，提高代码可读性和IDE支持
from langgraph.graph import StateGraph, START, END  # LangGraph核心组件：状态图、开始节点、结束节点
from langchain_openai import ChatOpenAI  # OpenAI模型的LangChain集成
from langchain_core.messages import HumanMessage  # LangChain消息类型

print("📚 库导入完成，准备构建邮件处理代理...")
print("🔧 即将使用的核心组件：")
print("   - StateGraph: 构建状态驱动的工作流")
print("   - ChatOpenAI: 调用 GPT 模型进行智能处理")
print("   - TypedDict: 定义严格的数据结构")

In [ ]:
# 🏗️ 定义邮件处理代理的状态结构

class EmailState(TypedDict):
    """
    邮件处理代理的状态数据结构
    
    这个类定义了代理在处理邮件过程中需要维护的所有状态信息
    """
    # 📧 原始邮件信息
    email: Dict[str, Any]  # 包含发件人、主题、正文等邮件完整信息
    
    # 🔍 垃圾邮件检测结果  
    is_spam: Optional[bool]  # 是否为垃圾邮件（True/False/None）
    spam_reason: Optional[str]  # 判定为垃圾邮件的原因说明
    
    # 🗂️ 邮件分类信息
    email_category: Optional[str]  # 邮件类别（如：商务、个人、紧急等）
    
    # ✍️ 回复草稿
    draft_response: Optional[str]  # 阿尔弗雷德为主人准备的回复草稿
    
    # 💬 对话历史记录
    messages: List[Dict[str, Any]]  # 存储处理过程中的LLM对话记录

print("✅ 邮件状态结构定义完成")
print("📋 状态字段说明：")
print("   - email: 原始邮件数据")
print("   - is_spam: 垃圾邮件判定结果") 
print("   - draft_response: 回复草稿")
print("   - messages: LLM对话历史") 

In [ ]:
# Initialize LLM
model = ChatOpenAI( model="gpt-4o",temperature=0)

class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    draft_response: Optional[str]
    messages: List[Dict[str, Any]]

# Define nodes
def read_email(state: EmailState):
    email = state["email"]
    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")
    return {}

def classify_email(state: EmailState):
    email = state["email"]
    
    prompt = f"""
As Alfred the butler of Mr wayne and it's SECRET identity Batman, analyze this email and determine if it is spam or legitimate and should be brought to Mr wayne's attention.

Email:
From: {email['sender']}
Subject: {email['subject']}
Body: {email['body']}

First, determine if this email is spam.
answer with SPAM or HAM if it's legitimate. Only return the answer
Answer :
    """
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    response_text = response.content.lower()
    print(response_text)
    is_spam = "spam" in response_text and "ham" not in response_text
    
    if not is_spam:
        new_messages = state.get("messages", []) + [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content}
        ]
    else :
        new_messages = state.get("messages", [])
    
    return {
        "is_spam": is_spam,
        "messages": new_messages
    }

def handle_spam(state: EmailState):
    print(f"Alfred has marked the email as spam.")
    print("The email has been moved to the spam folder.")
    return {}

def drafting_response(state: EmailState):
    email = state["email"]
    
    prompt = f"""
As Alfred the butler, draft a polite preliminary response to this email.

Email:
From: {email['sender']}
Subject: {email['subject']}
Body: {email['body']}

Draft a brief, professional response that Mr. Wayne can review and personalize before sending.
    """
    
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]
    
    return {
        "draft_response": response.content,
        "messages": new_messages
    }

def notify_mr_wayne(state: EmailState):
    email = state["email"]
    
    print("\n" + "="*50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print("\nI've prepared a draft response for your review:")
    print("-"*50)
    print(state["draft_response"])
    print("="*50 + "\n")
    
    return {}

# Define routing logic
def route_email(state: EmailState) -> str:
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"

# Create the graph
email_graph = StateGraph(EmailState)

# Add nodes
email_graph.add_node("read_email", read_email) # the read_email node executes the read_mail function
email_graph.add_node("classify_email", classify_email) # the classify_email node will execute the classify_email function
email_graph.add_node("handle_spam", handle_spam) #same logic 
email_graph.add_node("drafting_response", drafting_response) #same logic
email_graph.add_node("notify_mr_wayne", notify_mr_wayne) # same logic


In [ ]:
# Add edges
email_graph.add_edge(START, "read_email") # After starting we go to the "read_email" node

email_graph.add_edge("read_email", "classify_email") # after_reading we classify

# Add conditional edges
email_graph.add_conditional_edges(
    "classify_email", # after classify, we run the "route_email" function"
    route_email,
    {
        "spam": "handle_spam", # if it return "Spam", we go the "handle_span" node
        "legitimate": "drafting_response" # and if it's legitimate, we go to the "drafting response" node
    }
)

# Add final edges
email_graph.add_edge("handle_spam", END) # after handling spam we always end
email_graph.add_edge("drafting_response", "notify_mr_wayne")
email_graph.add_edge("notify_mr_wayne", END) # after notifyinf Me wayne, we can end  too


In [9]:
# Compile the graph
compiled_graph = email_graph.compile()

In [10]:
 # Example emails for testing
legitimate_email = {
    "sender": "Joker",
    "subject": "Found you Batman ! ",
    "body": "Mr. Wayne,I found your secret identity ! I know you're batman ! Ther's no denying it, I have proof of that and I'm coming to find you soon. I'll get my revenge. JOKER"
}

spam_email = {
    "sender": "Crypto bro",
    "subject": "The best investment of 2025",
    "body": "Mr Wayne, I just launched an ALT coin and want you to buy some !"
}


In [ ]:
from langfuse.langchain import CallbackHandler

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()

# Process legitimate email
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke(
    input = {
        "email": legitimate_email,
        "is_spam": None,
        "draft_response": None,
        "messages": []
        },
    config={"callbacks": [langfuse_handler]}
)

# Process spam email
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke(
    input = {
        "email": spam_email,
        "is_spam": None,
        "draft_response": None,
        "messages": []
        },
    config={"callbacks": [langfuse_handler]}
) 

### 追踪结构

Langfuse 会记录包含若干 **span（跨度）** 的**trace（追踪）**，每个 span 代表代理逻辑中的一个步骤。本例中的追踪包含整体运行以及如下子跨度：
- 工具调用（get_weather）
- LLM 调用（使用 'gpt-4o' 的 Responses API）

你可以检查这些记录以精确了解时间消耗、令牌使用量等：

![Langfuse 中的追踪树](https://langfuse.com/images/cookbook/example-langgraph-evaluation/trace-tree.png)

_[前往该追踪](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/3dd76e4b-980c-40eb-ae6d-ba9db5f6a349?timestamp=2025-03-20T14%3A56%3A16.665Z&display=details&observation=22b11054-93a8-4ff9-b862-babfcee906ec)_

## 在线评估

在线评估指在真实线上环境（生产环境的实际使用中）对代理进行评估。这需要对真实用户交互进行持续监控与结果分析。

我们在此总结了多种评估技术的指南：[链接](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)。

### 生产环境常见监控指标

1. **成本（Costs）**：埋点会记录令牌用量，你可按每个令牌的价格估算成本。
2. **延迟（Latency）**：观察完成每个步骤或整次运行所需的时间。
3. **用户反馈（User Feedback）**：用户可直接提供反馈（如点赞/点踩）以帮助迭代与修正代理。
4. **LLM 评审（LLM-as-a-Judge）**：使用额外的 LLM 近实时评估代理输出（如检测毒性或正确性）。

下面展示这些指标的示例。

#### 1. 成本（Costs）

下图展示了 `gpt-4o` 调用的用量，可据此识别高成本步骤并优化代理。

![成本](https://langfuse.com/images/cookbook/example-langgraph-evaluation/gpt-4o-costs.png)

_[前往该追踪](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/3dd76e4b-980c-40eb-ae6d-ba9db5f6a349?timestamp=2025-03-20T14%3A56%3A16.665Z&display=details&observation=22b11054-93a8-4ff9-b862-babfcee906ec)_

#### 2. 延迟（Latency）

还可以查看完成每个步骤所需的时间。如下例所示，整个运行约 3 秒，你可以细分到各步骤。此举有助于识别瓶颈并优化代理。

![延迟](https://langfuse.com/images/cookbook/example-langgraph-evaluation/agent-latency.png)

_[前往该追踪](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/3dd76e4b-980c-40eb-ae6d-ba9db5f6a349?timestamp=2025-03-20T14%3A56%3A16.665Z&display=timeline)_

#### 3. 用户反馈（User Feedback）

如果你的代理嵌入在用户界面中，可以采集用户的直接反馈（例如在聊天界面中的点赞/点踩）。 

In [ ]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="langgraph-request") as span:
    # ... LangGraph execution ...
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was correct, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="langgraph-request") as span:
    # ... LangGraph execution ...
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )
 
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined-trace-id", # Needs to be a valid trace id format (see docs)
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

用户反馈随后会被 Langfuse 捕获：

![Langfuse 中捕获的用户反馈](https://langfuse.com/images/cookbook/example-langgraph-evaluation/user-feedback.png)

#### 4. 自动化的 LLM 评审打分（LLM-as-a-Judge）

LLM-as-a-Judge 提供了一种自动评估代理输出的方法。你可以配置一个独立的 LLM 调用，用于评估输出的正确性、毒性、风格或其他你关心的指标。

**工作流程：**
1. 定义一个**评估模板**，例如“检查文本是否含有毒性”。
2. 指定用于评审的模型（judge-model），例如 `gpt-4o-mini`。
2. 每当代理生成输出时，将其与模板一起传给“评审”LLM。
3. 评审 LLM 给出评分或标签，并将结果记录到可观测性平台。

Langfuse 示例：

![LLM 评审模板](https://langfuse.com/images/cookbook/integration_openai-agents/evaluator-template.png)
![LLM 评审器](https://langfuse.com/images/cookbook/integration_openai-agents/evaluator.png)

In [ ]:
# Process spam email
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke(
    input = {
        "email": spam_email,
        "is_spam": None,
        "draft_response": None,
        "messages": []
        },
    config={"callbacks": [langfuse_handler]}
) 

可以看到，该示例的答案被评审为“无毒性（not toxic）”。

![LLM 评审得分示例](https://langfuse.com/images/cookbook/example-langgraph-evaluation/llm-as-a-judge-score.png)

#### 5. 可观测性指标总览

所有上述指标都可以在统一的仪表盘中可视化。这样你可以快速查看代理在多次会话中的表现，并随时间跟踪质量指标。

![可观测性指标总览](https://langfuse.com/images/cookbook/integration_openai-agents/dashboard-dark.png)

## Offline Evaluation

Online evaluation is essential for live feedback, but you also need **offline evaluation**—systematic checks before or during development. This helps maintain quality and reliability before rolling changes into production.

### Dataset Evaluation

In offline evaluation, you typically:
1. Have a benchmark dataset (with prompt and expected output pairs)
2. Run your agent on that dataset
3. Compare outputs to the expected results or use an additional scoring mechanism

Below, we demonstrate this approach with the [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset), which contains questions and expected answers.

In [ ]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

接下来，我们在 Langfuse 中创建一个数据集实体以追踪运行；随后将数据集中的每条记录添加到系统中。

In [ ]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_langgraph-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

In [ ]:
df_30 = df.sample(30) # For this example, we upload only 30 dataset questions

for idx, row in df_30.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Langfuse 中的数据集条目](https://langfuse.com/images/cookbook/example-langgraph-evaluation/example-dataset.png)

#### 在数据集上运行代理

首先，构建一个使用 OpenAI 模型回答问题的简易 LangGraph 代理。 

In [15]:
from typing import Annotated
 
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from typing_extensions import TypedDict
 
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
 
class State(TypedDict):
    messages: Annotated[list, add_messages]
 
graph_builder = StateGraph(State)
 
llm = ChatOpenAI(model = "gpt-4.5-preview")

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}
 
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")

graph = graph_builder.compile()

Then, we define a helper function `my_agent()` that:
1. Creates a Langfuse trace 
2. Fetches the `langfuse_handler_trace` to instrument the LangGraph execution. 
3. Runs our agent and passing `langfuse_handler_trace` to the invocation. 

In [ ]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

class State(TypedDict):
    messages: Annotated[list, add_messages]
 
graph_builder = StateGraph(State)
llm = ChatOpenAI(model = "gpt-4o")
langfuse = get_client()

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}
 
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

def my_agent(question, langfuse_handler):
    
    # Create a trace via Langfuse spans and use Langchain within it
    with langfuse.start_as_current_span(name="my-langgraph-agent") as root_span:
    
        # Step 2: LangChain processing
        response = graph.invoke(
        input={"messages": [HumanMessage(content = question)]},
        config={"callbacks": [langfuse_handler]}
        )
    
        # Update trace output
        root_span.update_trace(
            input=question,
            output=response["messages"][1].content)
        
        print(question)
        print(response["messages"][1].content)

    return response["messages"][1].content

Finally, we loop over each dataset item, run the agent, and link the trace to the dataset item. We can also attach a quick evaluation score if desired.

In [ ]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler
 
# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()
langfuse = get_client()

dataset = langfuse.get_dataset('qa-dataset_langgraph-agent')

for item in dataset.items:
    # Use the item.run() context manager for automatic trace linking
    with item.run(
        run_name="run_gpt-4o",
        run_description="My first run",
        run_metadata={"model": "gpt-4o"},
    ) as root_span:
        # All operations within this block are part of the trace for this dataset item
        
        # Call your application logic - this can use any combination of decorators, 
        # context managers, or manual observations
        with langfuse.start_as_current_generation(
            name="llm-call",
            model="gpt-4o",
            input=item.input
        ) as generation:
            # Your LLM application logic here
            output = my_agent(str(item.input), langfuse_handler)
            generation.update(output=output)
 
        # Optionally, score the result against the expected output
        root_span.score_trace(
            name="user-feedback",
            value=1,
            comment="This is a comment",  # optional, useful to add reasoning
        )
 
# Flush the langfuse client to ensure all data is sent to the server at the end of the experiment run
langfuse.flush()

You can repeat this process with different agent configurations such as:
- Models (gpt-4o-mini, o1, etc.)
- Prompts
- Tools (search vs. no search)
- Complexity of agent (multi agent vs single agent)

Then compare them side-by-side in Langfuse. In this example, I did run the agent 3 times on the 30 dataset questions. For each run, I used a different OpenAI model. You can see that amount of correctly answered questions improves when using a larger model (as expected). The `correct_answer` score is created by an [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) that is set up to judge the correctness of the question based on the sample answer given in the dataset.

![Dataset run overview](https://langfuse.com/images/cookbook/example-langgraph-evaluation/dataset_runs.png)
![Dataset run comparison](https://langfuse.com/images/cookbook/example-langgraph-evaluation/dataset-run-comparison.png)
